### HH парсер
* Парсит вакансии по ключевым словам в СПБ и МСК(удаленная занятость)
* может автоматически отправлять резюме, если оно еще не отправлялось

In [46]:
import requests, json, os
import pandas as pd
import numpy as np
import warnings
from datetime import datetime as dt
warnings.filterwarnings('ignore')
import webbrowser
from bs4 import BeautifulSoup
from urllib.request import urlopen
from collections import defaultdict
from tqdm import tqdm_notebook
from datetime import datetime as dt
import time
import seaborn as sns

import matplotlib.pyplot as plt
plt.style.use('seaborn-ticks')

from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from bs4 import BeautifulSoup 
from tqdm import tqdm_notebook
from wordcloud import WordCloud, STOPWORDS
from nltk.stem.snowball import SnowballStemmer

In [75]:
def collect_hh_vacancies(area, query, pages):    
    #######################################################################################
    def getPage(area, page, query):
        """
        Создаем метод для получения страницы со списком вакансий.
        Аргументы:
            page - Индекс страницы, начинается с 0. Значение по умолчанию 0, т.е. первая страница
        """

        # Справочник для параметров GET-запроса
        params = {
            'text': f'NAME:{query}', 
            'area': f'{area}', 
            'page': page, 
            'per_page': 100
            }
        req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
        data = req.content.decode() 
        req.close()
        return data

    def flatten_json(json_file):
        L= []
        for k, v in i.items():
            if isinstance(v, dict):
                for k2, v2 in v.items():
                    if not(isinstance(v2, dict)):
                        L.append((f'{k}.{k2}', v2))
                    else:
                        for k3, v3 in v2.items():
                            L.append((f'{k}.{k2}.{k3}', v3))
                        
            else:
                L.append((k, v))        
        _df = pd.DataFrame.from_records(L).set_index(0)
        _df.columns = _df.loc['id'].values    
        return _df.iloc[1:].T  
    ###########################################################################################
    
    # список с коллекцией выгруженных вакансий
    L_rows=[]    

    for page_idx in tqdm_notebook(pages):

        # выгружаем
        json_spb = json.loads(getPage(area=area, page=page_idx,query=query))     
        
        try:
            
            for i in json_spb['items']: 
                
                # коллекционируем
                L_rows.append(flatten_json(i))
                
                # спим, чтобы сайт не ограничил доступ
                time.sleep(.1)   
        except:
            pass  
    
    # собираем датафрейм
    df = pd.concat(L_rows)
    for col in df.columns:
        try:
            df[col] = df[col].astype(np.float32)
        except:
            pass
    df['published_at'] = pd.to_datetime(df['published_at'])
    return df.sort_values('published_at', ascending = False).reset_index(drop = True)

In [76]:
df1 = collect_hh_vacancies(area=113,query='аналитик', pages=np.arange(10))
df2 = collect_hh_vacancies(area=113,query='data scientist', pages=np.arange(10))
df3 = collect_hh_vacancies(area=113,query='bi', pages=np.arange(10))

### накладываем условия
* санкт-петербург
* другие города, если удаленка
* python, sql, bi в описании требований
* вакансии за последние 2 дня
* ранжирование вакансий по требованиям

In [191]:
# собираем датафрейм
df = pd.concat([df1, df2, df3], 0).reset_index(drop = True).dropna(subset = ['snippet.requirement'])

# требования
req = df['snippet.requirement'].str.lower()
mask_spb = df['area.name'].isin(['Санкт-Петербург'])
mask_udalenka = df['schedule.name'] == 'Удаленная работа'
mask_python = req.apply(lambda x: 'python' in x)
mask_sql = req.apply(lambda x: 'sql' in x)
mask_bi = req.apply(lambda x: 'bi' in x)

# используем колонки
cols_to_use = ['name', 'published_at', 'salary.from', 'salary.to',\
               'employer.name', 'snippet.requirement',\
               'snippet.responsibility', 'schedule.name', 'alternate_url']
ru_cols_to_use=['название', 'дата_публикации', 'зп_от', 'зп_до', 'компания', 'требования',\
                'обязанности', 'график_работы', 'url']

# финальная таблица(вакансии за последний день)
df_final = df[(mask_spb|(~mask_spb&mask_udalenka)) & (mask_python|mask_sql|mask_bi)]\
               .reset_index(drop = True)\
               [cols_to_use]\
               .rename(columns = dict(zip(cols_to_use, ru_cols_to_use)))
df_final['зарпалата'] = df_final[['зп_от', 'зп_до']].mean(1)
df_final = df_final.drop(['зп_от', 'зп_до'], 1)

# дата публикации
datetimes = pd.to_datetime(df_final['дата_публикации'].dt.date)
# последняя публикация
date_today = pd.to_datetime(dt.now().date())
df_final = df_final[((date_today - datetimes) / np.timedelta64(1, 'D'))<=2]                   
df_final = df_final[~df_final.duplicated()]          

# ранжирование вакансий по требованиям
req = df_final['требования'].str.lower()
sql_mask = req.str.contains('sql').astype(int)
pandas_mask = req.str.contains('pandas').astype(int)
python_mask = req.str.contains('python').astype(int)
df_final = df_final.loc[(python_mask+pandas_mask+sql_mask).sort_values(ascending = False).index]

# выбрасываем middle и senior
mask_notmiddle = df_final['название'].str.lower().apply(lambda x: 'middle' not in x)
mask_notsenior = df_final['название'].str.lower().apply(lambda x: 'senior' not in x)
mask_notsenior_v2 = df_final['название'].str.lower().apply(lambda x: 'старший' not in x)
df_final = df_final[(mask_notmiddle)&(mask_notsenior)&(mask_notsenior_v2)].reset_index(drop = True)   
df_final = df_final[~df_final['компания'].isin(['Филберт', 'Softwise'])]

In [192]:
df_final

,название,дата_публикации,компания,требования,обязанности,график_работы,url,зарпалата
0,Аналитик BI,2021-06-19 21:00:42+03:00,ДИВАН.РУ,"Знания основ sql, python+pandas (будет большим...",Сбор требований и правила расчета ключевых пок...,Удаленная работа,https://hh.ru/vacancy/44494198,NaN
1,Data Analyst,2021-06-19 12:58:15+03:00,MacBuildServer,Опыт работы с табличными данными. Навыки работ...,Изучать бизнес-данные и продумывать оптимальны...,Удаленная работа,https://hh.ru/vacancy/44904368,35000.0
2,Аналитик данных / Бизнес-аналитик,2021-06-18 15:53:23+03:00,Мотифай,"Образование экономическое, математическое, тех...","Анализ продукта (подготовка, проведение и анал...",Полный день,https://hh.ru/vacancy/45191213,170000.0
3,Аналитик,2021-06-19 18:23:05+03:00,Паблишер,"Понимание основ статистики, теории вероятносте...",Повышать качество принимаемых решений за счёт ...,Полный день,https://hh.ru/vacancy/45508070,NaN
4,Продуктовый аналитик,2021-06-19 15:21:55+03:00,"Лента, федеральная розничная сеть","Имеешь опыт работы в рознице, онлайн-проектах....",Проверять продуктовые гипотезы (проводить эксп...,Удаленная работа,https://hh.ru/vacancy/44913116,NaN
5,Аналитик BI,2021-06-18 22:44:25+03:00,"Бристоль, сеть магазинов",Хорошее знание SQL. Знание одного из языков пр...,Сбор требований бизнес-заказчиков и реализация...,Удаленная работа,https://hh.ru/vacancy/45574951,NaN
6,Аналитик по RPD,2021-06-18 16:59:47+03:00,GS Group,Высшее образование. Продвинутое знание статист...,"Анализ данных о потреблении услуг ""Триколор ТВ...",Полный день,https://hh.ru/vacancy/45502591,NaN
7,Продуктовый аналитик,2021-06-18 16:37:31+03:00,DIGINETICA,Хорошее понимание математической статистики. П...,Ядро продукта – ансамбль алгоритмов на базе ма...,Удаленная работа,https://hh.ru/vacancy/44990449,NaN
8,Продуктовый аналитик,2021-06-18 13:32:11+03:00,Межрегиональный ТранзитТелеком (ОАО МТТ),"Знать SQL и уметь работать с базами данных, вл...",Собрать данные. Рассчитать метрики. Сформирова...,Удаленная работа,https://hh.ru/vacancy/43639827,NaN
9,Data Scientist,2021-06-19 16:05:08+03:00,DIGINETICA,Уметь писать готовый к production код (Python/...,Разрабатывать модели схожести запросов и товар...,Удаленная работа,https://hh.ru/vacancy/44487032,NaN
